In [0]:
from azure.storage.blob import BlobServiceClient
from pyspark.sql.functions import col, decode, split, element_at,udf,input_file_name,regexp_extract,expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, LongType

In [0]:

expected_schema = StructType([
    StructField("operation", StringType(), True),
    StructField("transaction_id", StringType(), True),
    StructField("relation_id", StringType(), True),
    StructField("a360_record_id", StringType(), True),
    StructField("process_time", TimestampType(), True),
    StructField("status", IntegerType(), True),
    StructField("input", StringType(), True),  # Contains nested JSON as a string
    StructField("exception_description", StringType(), True),
    StructField("error_status", StringType(), True),
    StructField("a360_file_id", StringType(), True),
    StructField("file_size", LongType(), True),
    StructField("s_md5", StringType(), True),
    StructField("s_sha256", StringType(), True),
    StructField("timestamp", TimestampType(), True),  # may be used as process_time
    StructField("filename", StringType(), True),
    StructField("submission_folder", StringType(), True),
    StructField("file_hash", StringType(), True)
])

In [0]:
#### Set up Auto Loader job

sas_token = dbutils.secrets.get(scope="ingest00-meta002-sbox", key="ARIATD-SAS-TOKEN")
storage_account_name = "a360c2x2555dz"
container_name = "dropzone"
sub_dir = "ARIATD/response"

input_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{sub_dir}"

spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)

# schema_location = "/mnt/autoLoaderSchema/ARMJOH/response/read_stream"

# Define a file regex that matches files ending with .rsp
file_regex = ".*\\.rsp$"

output_container_name = "silver" 

output_storage_account_name = "ingest00curatedsbox"

output_subdir_amalgamated_responses = "ARIADM/ARM/response/TD/amalgamated_responses"
amalgamated_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_amalgamated_responses}"



output_subdir_input_upload = "ARIADM/ARM/response/TD/input_upload"
input_upload_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_input_upload}"


output_subdir_create_record_upload = "ARIADM/ARM/response/TD/create_record"
create_record_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_create_record_upload}"


output_subdir_upload_file_upload = "ARIADM/ARM/response/TD/upload_file"
upload_file_responses_path = f"wasbs://{output_container_name}@{output_storage_account_name}.blob.core.windows.net/{output_subdir_upload_file_upload}"

In [0]:
# Service principal credentials
client_id = dbutils.secrets.get("ingest00-meta002-sbox", "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get("ingest00-meta002-sbox", "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get("ingest00-meta002-sbox", "SERVICE-PRINCIPLE-TENANT-ID")

# Storage account names

checkpoint_storage = "ingest00xcuttingsbox"


# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


spark.conf.set(f"fs.azure.account.auth.type.{output_storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{output_storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{output_storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{output_storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{output_storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")



check_point_path = "abfss://db-rsp-checkpoint@ingest00xcuttingsbox.dfs.core.windows.net/ARMTD/RSP/"

schema_location = "abfss://db-rsp-checkpoint@ingest00xcuttingsbox.dfs.core.windows.net/ARMTD/RSP/schema"

In [0]:
### Run autoloader read stream


df = (spark.readStream.format("cloudFiles")
      .schema(expected_schema)
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", schema_location)
    .option("multiline", "true")
    .option("cloudFiles.schemaEvolutionMode", "none")
    .option("checkpointLocation", f"{check_point_path}/rsp_readStream")
    .load(input_path)
    .select( "*",col("_metadata.file_path").alias("_file_path"),
             col("_metadata.file_modification_time").alias("_file_modification_time")
)
    .withColumn("id", expr("uuid()")))

In [0]:
### 4 write streams to Silver container in blob storage

output_sas = dbutils.secrets.get(scope="ingest00-meta002-sbox", key="CURATED-SAS-TOKEN")

spark.conf.set(
    f"fs.azure.sas.{output_container_name}.{output_storage_account_name}.blob.core.windows.net",
    output_sas
)

### first save the complete table

df_complete = df.writeStream \
    .queryName("amalgamated") \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/amalgamated")\
            .outputMode("append")\
                    .start(amalgamated_responses_path)               

In [0]:
### Create Record table
df_create_upload = df.select("id","operation","transaction_id","relation_id","a360_record_id","process_time","status","input","exception_description","error_status","_file_path","_file_modification_time"
).filter(col("operation")=="create_record")\
    .writeStream \
    .queryName("create_record") \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/create_record")\
            .outputMode("append")\
                .start(create_record_responses_path)

In [0]:
### upload file table
df_upload = df.select("id","operation","transaction_id","relation_id","a360_record_id","process_time","status","input","exception_description","error_status","a360_file_id","file_size","s_md5","s_sha256","_file_path","_file_modification_time"
).filter(col("operation")=="upload_new_file")\
    .writeStream \
    .queryName("upload_new_file") \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/upload_file")\
            .outputMode("append")\
                .start(upload_file_responses_path)

In [0]:
### Input upload table

df_input_upload = df.select("id","operation","timestamp","status","exception_description","error_status","filename","submission_folder","file_hash","_file_path","_file_modification_time"
).filter(col("operation")=="input_upload")\
    .writeStream \
    .queryName("input_upload") \
    .format("delta")\
        .option("checkpointLocation", f"{check_point_path}/input_upload")\
            .outputMode("append")\
                .start(input_upload_responses_path)

In [ ]:
# Wait 15 minutes for any stream to stop itself, or auto stop after 15mins.

try:
    spark.streams.awaitAnyTermination(900)
finally:
    # Stop all streams
    df_complete.stop()
    df_input_upload.stop()
    df_create_upload.stop()
    df_upload.stop()
    print("All streams have been stopped")


df_input_upload = spark.read.format("delta").load(input_upload_responses_path)
 
# # Read the response data for create_record_upload
df_create_record_upload = spark.read.format("delta").load(create_record_responses_path)
 
# # Read the response data for upload_file
df_upload_file_upload = spark.read.format("delta").load(upload_file_responses_path) 
 
# Read the response data for df_amalgamated_responses
df_amalgamated_responses = spark.read.format("delta").load(amalgamated_responses_path)

print(f"Records in df_input_upload: {df_input_upload.count()}")
print(f"Records in df_create_record_upload: {df_create_record_upload.count()}")
print(f"Records in df_upload_file_upload: {df_upload_file_upload.count()}")
print(f"Records in df_amalgamated_responses: {df_amalgamated_responses.count()}")

In [0]:
dbutils.notebook.exit("Notebook execution completed successfully.")